In [ ]:
import os
from notion_client import Client
from dotenv import load_dotenv


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the secrets
secret_key = os.getenv('SECRET_KEY')
database_url = os.getenv('DATABASE_URL')

print(f'Secret Key: {secret_key}')
print(f'Database URL: {database_url}')


In [ ]:
notion = Client(auth=os.environ["NOTION_TOKEN"])

In [ ]:
from pprint import pprint

list_users_response = notion.users.list()
pprint(list_users_response)
